In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

test = pd.read_csv('../input/test.csv')
train = pd.read_csv('../input/train.csv')

In [ ]:
print(test.shape)
print(train.shape)
print(test.describe())
print(train.describe())
print(test.isnull().sum())
print(train.isnull().sum())
#欠損はtrainもtestもなし

In [ ]:
#store は　1-10
#itemは1-50
#dateは2013/1/1-2017/12/31の日時データ
print(train["store"].unique())
print(train["item"].unique())
print(train["date"].unique())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.xlim(train.item.min(), train.item.max()*1.1)
sns.boxplot(x=train.item)
plt.figure(figsize=(10,4))
plt.xlim(train.sales.min(), train.sales.max()*1.1)
sns.boxplot(x=train.sales)
#髭の長さは箱の長さの1.5倍までらしく、それをはみ出たものを外れ値としてプロットしてる。
#つまり第一四分位点から第三四分位点までの長さ×1.5

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(train["sales"])

In [ ]:
from fbprophet import Prophet
from multiprocessing import Pool
import multiprocessing as multi

#prophetに突っ込む準備
train2=train.rename(columns={"date":"ds","sales":"y"})

def use_prophet(ids):
    shop_id,item_id=ids
    #print(shop_id,item_id)
    temp=train2[(train2["store"]==shop_id) & (train2["item"]==item_id)][["ds","y"]]
    if len(temp.index)==0:
        pred=0
    elif len(temp.index)==1:
        pred=temp["y"].tail(1)
    else:
        #train["cap"]=2500
        #growth="logistic",
        model=Prophet(yearly_seasonality=True)
        model.fit(temp)
        future = model.make_future_dataframe(periods=90)
        #future["cap"]=2500
        forecast = model.predict(future)
        pred=forecast["yhat"].tail(90)
    return pred

id_list=test[["store","item"]].values.tolist()
id_list2=[]
for i in id_list:
    if i not in id_list2:
        id_list2.append(i)
        
p = Pool(multi.cpu_count())
result=p.map(use_prophet, id_list2)
p.close()

In [ ]:
result_list=[]
for j in result:
    result_list.extend(j.values.tolist())

submission=pd.DataFrame({
    "id" : test["id"],
    "sales" : result_list
})

In [ ]:
submission.to_csv("submission_prophet.csv", index=False)